In [3]:
import pandas as pd
import cloudpickle
import numpy as np

In [6]:
df = cloudpickle.load(open("pickles/cleaned_forum_data.pkl", "rb" ) )

totest=df.shape[0]

asktxt = df['askertxt'].as_matrix()[:totest]
doctxt = df['doctortxt'].as_matrix()[:totest]

diag = cloudpickle.load(open("pickles/doctortext_labels.pkl", "rb" ) )[:totest,:]
y=np.asarray(diag.todense()).squeeze()
# y=y[:,y.sum(axis=0)>50]

In [7]:
y.shape

(63846, 72)

In [9]:
## import gensim, string
from gensim.models.doc2vec import LabeledSentence
from collections import namedtuple
import string

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

alldocs = []  # will hold all docs in original order
alltxts = []
for line_no, line in enumerate(asktxt):
#     words = my_normalize_nostem(line)
    words = line.lower().translate(string.maketrans(string.punctuation, ' '*len(string.punctuation))).split()
    tags = [line_no] # `tags = [tokens[0]]` would also work at extra memory cost
    alldocs.append(LabeledSentence(words, tags))
    alltxts.append(words)
#     alltxts.append([stemmer.stem(t) for t in words])
    
# for line_no, line in enumerate(doctxt):
# #     words = my_normalize_nostem(line)
#     words = line.lower().translate(string.maketrans(string.punctuation, ' '*len(string.punctuation))).split()
#     tags = [len(asktxt)+line_no] # `tags = [tokens[0]]` would also work at extra memory cost
#     alldocs.append(LabeledSentence(words, tags))

# doc_list = alldocs[:]  # for reshuffling per pass

from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"


# my current doc2vec of choice
# d2v_vectorizer = Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=3, workers=cores)

# #     other options
# #     'd2v-sg-concat-200', Doc2Vec(dm=1, dm_concat=1, size=200, window=5, negative=5, hs=0, min_count=2, workers=cores)
# #     'd2v-cbow', Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=2, workers=cores)
# #     'd2v-sg-mean-100', Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=5, hs=0, min_count=2, workers=cores)
# #     'd2v-sg-mean-200', Doc2Vec(dm=1, dm_mean=1, size=200, window=10, negative=5, hs=0, min_count=2, workers=cores)

# # speed setup by sharing results of 1st model's vocabulary scan
# d2v_vectorizer.build_vocab(alldocs)  # PV-DM/concat requires one special NULL word so it serves as template

# %time d2v_vectorizer.train(alldocs)
    
# vec_represent = %time np.array([d2v_vectorizer.infer_vector(doc.words) for doc in alldocs])



In [ ]:
# import nltk
# from nltk.stem.snowball import SnowballStemmer
# stemmer = SnowballStemmer("english")

# remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
# # remove_punctuation_map = string.maketrans(string.punctuation, ' '*len(string.punctuation))
# ## First function that creates the tokens
# def stem_tokens(tokens):
#     return [stemmer.stem(item.translate(remove_punctuation_map)) for item in tokens]

# ## Function that incorporating the first function, converts all words into lower letters and removes puctuations maps (previously specified)
# def normalize(text):
#     return stem_tokens(nltk.word_tokenize(text.lower()))


In [1]:



# or use boring vectorizers with different architecture
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
txt_list = np.array(alltxts)
tfidf_vec=TfidfVectorizer(analyzer=lambda x: x, stop_words='english', ngram_range=(1,3), max_df=0.8, max_features=10000, min_df=0.01)
#     'bagofwords',CountVectorizer(analyzer=lambda x: x, stop_words='english', max_df=0.4, max_features=8000, min_df=0.01)
%time tfidf_vec.fit(txt_list)
vec_represent=tfidf_vec.transform(txt_list)

In [6]:
import cloudpickle
with open("pickles/forum_vectorizer.pkl", 'wb') as handle:
    cloudpickle.dump(tfidf_vec, handle)
with open("pickles/forum_vecrepresent.pkl", 'wb') as handle:
    cloudpickle.dump(vec_represent, handle)

In [4]:
tfidf_vec.fit(np.array(['the','boy','was','tall']))

TfidfVectorizer(analyzer=<function <lambda> at 0x7efbca280b90>, binary=False,
        decode_error=u'strict', dtype=<type 'numpy.int64'>,
        encoding=u'utf-8', input=u'content', lowercase=True, max_df=0.8,
        max_features=10000, min_df=0.01, ngram_range=(1, 3), norm=u'l2',
        preprocessor=None, smooth_idf=True, stop_words='english',
        strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [5]:
tfidf_vec.transform(np.array(['the','boy','was','short']))

<4x10 sparse matrix of type '<type 'numpy.float64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [15]:
txt_list=np.array(alltxts)
txt_list.shape

(63846,)

In [16]:
vectorized = %time tfidf_vec.fit(txt_list).transform(txt_list)

CPU times: user 20.7 s, sys: 724 ms, total: 21.5 s
Wall time: 21.4 s


In [17]:
vectorized.shape

(63846, 892)

In [21]:
query = tfidf_vec.transform(np.array([['the','boy','was','short']]))

In [22]:
query.shape

(1, 892)